# 転移学習

### いつもの設定

In [1]:
from type_hint import *
from import_str import importstr
from log_conf import logging
log = logging.getLogger('nb')

# log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

# running everything app
def run(app, *argv):
    argv = list(argv)
    log.info('Running: {}({!r}).main()'.format(app, argv))
    print("*app.rsplit('.', 1) : ", *app.rsplit('.', 1))

    app_cls = importstr(*app.rsplit('.', 1)) # __import__を実行
    app_cls(argv).main()

    log.info("Finished: {}.({!r}).main()".format(app, argv))

module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Wiki_AI/..
module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Wiki_AI/..
module_parent_dir /Users/inoueshinichi/Desktop/MyGithub/Wiki_AI/..


### 前処理

In [35]:
# 画像前処理動作を確認
def preprocessImage():
    
    # 画像読み込み
    image_file_path = "./data/goldenretriever-3724972_640.jpg"
    img = Image.open(image_file_path)

    # 原画像を表示
    plt.imshow(img)
    plt.show()

    # 前処理と処理後の画像を表示
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = ImageTransform(resize, mean, std)
    img_transformed = transform(img, 'train') # torch.Size([3, 224, 224])
    img_transformed_transposed = img_transformed.numpy().transpose((1,2,0))

    print("img_transformed_transposed :",  img_transformed_transposed)
    img_transformed_transposed_cliped = np.clip(img_transformed_transposed, 0, 1) # 0-1にクリップ
    print("img_transformed_transposed_cliped :",  img_transformed_transposed_cliped)
    plt.imshow(img_transformed_transposed_cliped)
    plt.show()

    return img_transformed # torch.Size()

# データセット

In [36]:
# アリとハチの画像へのファイルパスのリストを作成する
def make_datapath_list(phase='train'):

    rootpath = "./data/Pytorch/hymenoptera_data/"
    target_path = osp.join(rootpath + phase + '/**/*.jpg')
    print(target_path)

    # ファイルパスを取得
    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)

    for i, path in enumerate(path_list):
        print(str(i) + ": " +  path)

    return path_list

In [37]:
print(make_datapath_list())

./data/Pytorch/hymenoptera_data/train/**/*.jpg
0: ./data/Pytorch/hymenoptera_data/train/bees/2638074627_6b3ae746a0.jpg
1: ./data/Pytorch/hymenoptera_data/train/bees/507288830_f46e8d4cb2.jpg
2: ./data/Pytorch/hymenoptera_data/train/bees/2405441001_b06c36fa72.jpg
3: ./data/Pytorch/hymenoptera_data/train/bees/2962405283_22718d9617.jpg
4: ./data/Pytorch/hymenoptera_data/train/bees/446296270_d9e8b93ecf.jpg
5: ./data/Pytorch/hymenoptera_data/train/bees/1092977343_cb42b38d62.jpg
6: ./data/Pytorch/hymenoptera_data/train/bees/2704348794_eb5d5178c2.jpg
7: ./data/Pytorch/hymenoptera_data/train/bees/2358061370_9daabbd9ac.jpg
8: ./data/Pytorch/hymenoptera_data/train/bees/2861002136_52c7c6f708.jpg
9: ./data/Pytorch/hymenoptera_data/train/bees/266644509_d30bb16a1b.jpg
10: ./data/Pytorch/hymenoptera_data/train/bees/2470492904_837e97800d.jpg
11: ./data/Pytorch/hymenoptera_data/train/bees/2053200300_8911ef438a.jpg
12: ./data/Pytorch/hymenoptera_data/train/bees/2601176055_8464e6aa71.jpg
13: ./data/Pytorc

In [38]:
# アリとハチの画像データセット
class HymenopteraDataset(data.Dataset):

    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    # 必須 1)
    def __len__(self):
        return len(self.file_list)


    # 必須 2)
    def __getitem__(self, index):

        img_path = self.file_list[index]
        img = Image.open(img_path)

        img_transformed = self.transform(img, self.phase) # torch.Size([3, 224, 224])

        if self.phase == 'train':
            label = img_path[30:34]
        elif self.phase == 'val':
            label = img_path[28:32]

        if label == 'ants':
            label = 0
        elif label == 'bees':
            label = 1

        return img_transformed, label

In [39]:
# データセット作成
def makeDataset():

    # 訓練
    train_list = make_datapath_list(phase='train')

    # 検証
    val_list = make_datapath_list(phase='val')

    # データセット
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = ImageTransform(resize, mean, std)
    train_dataset = HymenopteraDataset(file_list=train_list, transform=transform, phase='train')
    val_dataset = HymenopteraDataset(file_list=val_list, transform=transform, phase='val')

    # 動作確認
    index = 0
    print(train_dataset.__getitem__(index)[0].size()) # img_transformed
    print(train_dataset.__getitem__(index)[1])        # label

    print("Training Data: ")
    for train_index in range(0, train_dataset.__len__()):
        print("train" + "_" + str(train_index) +  " :" + str(train_dataset.__getitem__(train_index)[1]))

    return train_dataset, val_dataset

In [40]:
def dataLoader(train_dataset = None, val_dataset = None):

    # ミニバッチサイズ
    batch_size  = 32

    # DataLoader作成
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset, batch_size = batch_size, shuffle = True
    )

    val_dataloader = torch.utils.data.DataLoader(
        val_dataset, batch_size = batch_size, shuffle = True
    )

    # 辞書にまとめる
    datasetloaders_dict = { 'train': train_dataloader, 'val': val_dataloader }

    # 動作確認
    batch_iterator = iter(datasetloaders_dict['train']) # イテレータに変換
    inputs, labels = next(batch_iterator) # 1番目の要素を取り出す
    print(inputs.size())
    print(labels)

    return datasetloaders_dict

### VGG16モデルの出力層を変更

In [41]:
def modefiedModelVGG16():

    # 学習済みモデルVGG-16をロード
    use_pretained = True
    net = models.vgg16(pretrained=use_pretained)

    # 出力層の出力ユニットをアリとハチの2つに付け替える
    net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

    # 訓練モードに設定
    net.train()
    print('ネットワーク設定完了: 学習済み重みを読み込み、訓練モードに設定しました。')

    # 損失関数の設定
    criterion = nn.CrossEntropyLoss()

    """ 最適化手法の設定 """
    # 転移学習で学習させるパラメータを変数params_to_updateに格納する
    params_to_update = []

    # 学習させるパラメータ名
    update_param_names = ["classifier.6.weight", "classifier.6.bias"]

    # 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
    for name, param in net.named_parameters():
        if name in update_param_names:
            param.requires_grad = True
            params_to_update.append(param)
            print(name)
        else:
            param.requires_grad = False
    
    # params_to_updateの中身を確認
    print('-------------------')
    print(params_to_update)


    # Momentum SGD
    optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

    return criterion, optimizer, net # 損失関数, 最適化手法

# 学習と検証

In [42]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('--------')

        # epochごとの学習と検証ループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 訓練モードのモデル
            else:
                net.eval()  # 検証モードのモデル

            epoch_loss = 0.0   # epochの損失和
            epoch_corrects = 0 # epochの正解数 

            # 未学習時の検証性能を確かめるため、epoch=0の訓練では未学習のモデルを使用する
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloader_dict[phase]):

                # optimzerを初期化
                optimizer.zero_grad()

                # forward計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)              # 出力
                    loss = criterion(outputs, labels)  # 誤差
                    _, preds = torch.max(outputs, 1)   # 予測ラベル

                    # 訓練時はbackpropagation
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イテレーションの計算結果
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0) # 平均loss * バッチ数
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

# 実行

In [43]:
# 1) データセット作成
train_dataset, val_dataset = makeDataset()

# 2) データローダー
dataloader_dict = dataLoader(train_dataset, val_dataset)

# 3) 転移学習モデル(VGG-16), 損失関数, 最適化手法設定
criterion, optimizer, net = modefiedModelVGG16()

# 4) 学習と検証
num_epochs = 2
train_model(net, dataloader_dict, criterion, optimizer, num_epochs = num_epochs)

./data/Pytorch/hymenoptera_data/train/**/*.jpg
0: ./data/Pytorch/hymenoptera_data/train/bees/2638074627_6b3ae746a0.jpg
1: ./data/Pytorch/hymenoptera_data/train/bees/507288830_f46e8d4cb2.jpg
2: ./data/Pytorch/hymenoptera_data/train/bees/2405441001_b06c36fa72.jpg
3: ./data/Pytorch/hymenoptera_data/train/bees/2962405283_22718d9617.jpg
4: ./data/Pytorch/hymenoptera_data/train/bees/446296270_d9e8b93ecf.jpg
5: ./data/Pytorch/hymenoptera_data/train/bees/1092977343_cb42b38d62.jpg
6: ./data/Pytorch/hymenoptera_data/train/bees/2704348794_eb5d5178c2.jpg
7: ./data/Pytorch/hymenoptera_data/train/bees/2358061370_9daabbd9ac.jpg
8: ./data/Pytorch/hymenoptera_data/train/bees/2861002136_52c7c6f708.jpg
9: ./data/Pytorch/hymenoptera_data/train/bees/266644509_d30bb16a1b.jpg
10: ./data/Pytorch/hymenoptera_data/train/bees/2470492904_837e97800d.jpg
11: ./data/Pytorch/hymenoptera_data/train/bees/2053200300_8911ef438a.jpg
12: ./data/Pytorch/hymenoptera_data/train/bees/2601176055_8464e6aa71.jpg
13: ./data/Pytorc

train_24 :a/tr
train_25 :a/tr
train_26 :a/tr
train_27 :a/tr
train_28 :a/tr
train_29 :a/tr
train_30 :a/tr
train_31 :a/tr
train_32 :a/tr
train_33 :a/tr
train_34 :a/tr
train_35 :a/tr
train_36 :a/tr
train_37 :a/tr
train_38 :a/tr
train_39 :a/tr
train_40 :a/tr
train_41 :a/tr
train_42 :a/tr
train_43 :a/tr
train_44 :a/tr
train_45 :a/tr
train_46 :a/tr
train_47 :a/tr
train_48 :a/tr
train_49 :a/tr
train_50 :a/tr
train_51 :a/tr
train_52 :a/tr
train_53 :a/tr
train_54 :a/tr
train_55 :a/tr
train_56 :a/tr
train_57 :a/tr
train_58 :a/tr
train_59 :a/tr
train_60 :a/tr
train_61 :a/tr
train_62 :a/tr
train_63 :a/tr
train_64 :a/tr
train_65 :a/tr
train_66 :a/tr
train_67 :a/tr
train_68 :a/tr
train_69 :a/tr
train_70 :a/tr
train_71 :a/tr
train_72 :a/tr
train_73 :a/tr
train_74 :a/tr
train_75 :a/tr
train_76 :a/tr
train_77 :a/tr
train_78 :a/tr
train_79 :a/tr
train_80 :a/tr
train_81 :a/tr
train_82 :a/tr
train_83 :a/tr
train_84 :a/tr
train_85 :a/tr
train_86 :a/tr
train_87 :a/tr
train_88 :a/tr
train_89 :a/tr
train_90 :


  0%|          | 0/5 [00:00<?, ?it/s]

ネットワーク設定完了: 学習済み重みを読み込み、訓練モードに設定しました。
classifier.6.weight
classifier.6.bias
-------------------
[Parameter containing:
tensor([[ 0.0117,  0.0116,  0.0082,  ..., -0.0072,  0.0059, -0.0065],
        [-0.0071, -0.0131, -0.0117,  ..., -0.0079, -0.0070,  0.0085]],
       requires_grad=True), Parameter containing:
tensor([-0.0087,  0.0008], requires_grad=True)]
Epoch 1/2
--------


AttributeError: 'tuple' object has no attribute 'size'